### 01 packages

In [1]:
##########################################################################################
##########################################################################################

import matplotlib.pyplot as plt

from matplotlib.ticker import MultipleLocator, FormatStrFormatter

##########################################################################################
##########################################################################################

import numpy as np

import pandas as pd

##########################################################################################
##########################################################################################

import os,sys

import copy

import math

import random

import time

import datetime

##########################################################################################
##########################################################################################

import h3

import folium

from shapely.geometry import Point, Polygon

import geopandas as gp

##########################################################################################
##########################################################################################

import warnings

warnings.filterwarnings("ignore")

### 02 data

In [2]:
##########################################################################################
##########################################################################################

latitude = 40.7831

longitude = -73.9712

##########################################################################################
##########################################################################################

store_df = pd.read_excel('Grocery.xlsx')

store_df=store_df[['name','latitude', 'longitude']]

store_df=store_df.dropna()

store_df=store_df.reset_index(drop=True)

display(store_df)

##########################################################################################
##########################################################################################


,name,latitude,longitude
0,Trader Joe's,40.742090,-73.994121
1,Whole Foods Market,40.734826,-73.991205
2,Whole Foods Market,40.754553,-73.984657
3,Mulberry Market NYC,40.723252,-73.996098
4,Westside Market NYC,40.731843,-73.988655
...,...,...,...
145,3900 Grocery Inc,40.837761,-73.941662
146,Fischer Bros. & Leslie Glatt Kosher,40.778988,-73.983267
147,Grocery,40.735950,-73.992865
148,Grocery,40.735959,-73.992762


### 03 Manhattan Network

In [3]:
##########################################################################################
##########################################################################################

def Get_POLYGON(coords):
    
    if coords.type=='Polygon':
        
        return coords
    
    else:
        
        Score={i:coords[i].area for i in range(len(coords))}
        
        idx=max(Score, key=Score.get)
        
        return coords[idx]
    
##########################################################################################
##########################################################################################

Zone_data = gp.read_file('./NYC_Zones/geo_export_789b911c-2a6d-4092-a682-6c5e6ce53409.shp')

Zone_data=Zone_data[['borough','zone','geometry','location_i']]

Zone_data=Zone_data.loc[Zone_data['borough']=='Manhattan']

Zone_data=Zone_data.reset_index(drop=True)

##########################################################################################
##########################################################################################

Zone_data['Zone_id']=['Zone_'+str(i) for i in Zone_data.index]

Zone_data=Zone_data[['Zone_id','borough','zone','geometry','location_i']]

Islands=['Zone_19','Zone_20','Zone_21','Zone_38','Zone_47','Zone_48']

Zone_data=Zone_data.loc[~Zone_data['Zone_id'].isin(Islands)]

Zone_data=Zone_data.reset_index(drop=True)

##########################################################################################
##########################################################################################

Zone_data['Zone_id']=['Zone_'+str(i) for i in Zone_data.index]

Zone_data['geometry']=Zone_data.apply(lambda x:Get_POLYGON(x['geometry']),axis=1)

Zone_data['center'] = Zone_data.apply(lambda x:x['geometry'].centroid,axis=1)

Zone_data=Zone_data.rename(columns={'location_i':'LocationID'})

Zone_data[['LocationID']] = Zone_data[['LocationID']].astype(float)

Zone_data['type']=Zone_data.apply(lambda x:x['geometry'].type,axis=1)

##########################################################################################
##########################################################################################

Zone_geometry={}

for idx,row in Zone_data.iterrows():
    
    Zone_geometry[int(row['LocationID'])]=row['geometry']

Zone_geometry

##########################################################################################
##########################################################################################

network = folium.Map(location=[latitude, longitude], zoom_start=12,tiles='CartoDB positron')

##########################################################################################
##########################################################################################

folium.Choropleth(
    geo_data=Zone_data[['geometry']],
    fill_color='blue',
    fill_opacity=0.1,
    name='Zone').add_to(network)

network


### Hexagon network with lockers

In [4]:
Hexagons=list()

##########################################################################################
##########################################################################################

# Function to calculate the midpoint between two coordinates

def calculate_midpoint(coord1, coord2):
    
    return ((coord1[0] + coord2[0]) / 2, (coord1[1] + coord2[1]) / 2)

##########################################################################################
##########################################################################################

# The desired resolution for the H3 hexagon

resolution = 8

# Number of rings away from the original hexagon to get neighbors

num_rings_away = 20

##########################################################################################
##########################################################################################

# Generate the H3 hexagon at the specified location and resolution

h3_hexagon = h3.geo_to_h3(latitude, longitude, resolution)


# Get the neighbors of the hexagon

neighbors = h3.k_ring(h3_hexagon, num_rings_away)

##########################################################################################
##########################################################################################

midpoints = []

hexagons_midpoints={}

midpoints_hexagons={}

##########################################################################################
##########################################################################################

for hexagon in neighbors:

    # Get the center coordinates of the hexagon
    
    hexagon_vertice=h3.h3_to_geo_boundary(hexagon)
    
    hexagon_midpoints=list()
    
    for i in range(len(hexagon_vertice)):
        
        # Calculate the midpoint between the current vertex and the next vertex (to close the polygon)
        
        midpoint = calculate_midpoint(hexagon_vertice[i], hexagon_vertice[(i + 1) % len(hexagon_vertice)])
        
        hexagon_midpoints.append(midpoint)
    
    cover_count=0
    
    for polygon in Zone_geometry.values():
    
        # Check if the coordinate is covered by the polygon
        
        for midpoint in hexagon_midpoints:
            
            midpoint_obj=list(midpoint)[::-1]

            is_covered = polygon.contains(Point(midpoint_obj))

            cover_count+=is_covered
            
            if cover_count>=2:

                Hexagons.append(hexagon)
                
                midpoints+=hexagon_midpoints
                
                midpoints=list(set(midpoints))
                
                hexagons_midpoints[hexagon]=hexagon_midpoints
                
                ##########################################################################################
                ##########################################################################################

                if midpoint not in midpoints_hexagons.keys():

                    midpoints_hexagons[midpoint]=[hexagon]

                else:

                    midpoints_hexagons[midpoint].append(hexagon)

                

                break

##########################################################################################
##########################################################################################

hexagon_vertices = {hexagon:h3.h3_to_geo_boundary(hexagon) for hexagon in Hexagons}

##########################################################################################
##########################################################################################

network = folium.Map(location=[latitude, longitude], zoom_start=12,tiles='CartoDB positron')

for vertices in hexagon_vertices.values():
    
    folium.Polygon(locations=vertices, color='black', opacity=1, fill=False, fill_color='blue', fill_opacity=0.0).add_to(network)
    
for midpoint in midpoints:
    
    folium.Circle(
    radius=20,
    location=midpoint,
    color='red',
    fill=True,
    ).add_to(network)

    
network


   

### Hexagon network with grocerys

In [5]:
grocerys=list()

for idx,row in store_df.iterrows():
    
    grocery_location=Point(row.longitude,row.latitude)
    
    for polygon in Zone_geometry.values():
    
        # Check if the coordinate is covered by the polygon

        is_covered = polygon.contains(grocery_location)
        
        if is_covered:
            
            grocerys.append((row.longitude,row.latitude))
            
            break



##########################################################################################
##########################################################################################

network = folium.Map(location=[latitude, longitude], zoom_start=12,tiles='CartoDB positron')

for vertices in hexagon_vertices.values():
    
    folium.Polygon(locations=vertices, 
                   color='black', 
                   opacity=1, 
                   fill=False,
                   weight=1,).add_to(network)
    
for midpoint in midpoints:
    
    folium.Circle(
    radius=10,
    location=midpoint,
    color='red',
    fill=True,
    ).add_to(network)
    
for grocery in grocerys:
    
    folium.Circle(
    radius=20,
    location=grocery[::-1],
    color='green',
    fill=True,
    ).add_to(network)


network.save('01nework.html')
    
network